# Multi-Layers Perceptron
The goal of this notebook is to showcase how to code a multilayer perceptron in Python from scratch.
It makes use of the Perceptron algorithm we developped in the perceptron jupyter notebook with modification to use backpropagation!

We have two main class:
- Neuron: Is used to model one neuron, most of the computation happens there
- MultiLayerPerceptron: Is used to model the full neural networ. Here we only support fully connected neural network and in this particular notebook we only have one hidden layer and one neuron in the output layer.

There is still much to be improved here, like modeling a layer so that it is easier to compose neural network, but the goal right now was to showcase how backpropagation works.s

In [2]:
import random
import math

class Neuron():
    '''
        A conceptual Neuront hat can be trained using a 
        fit and predict methodology, without any library
    '''
    
    def __init__(self, position_in_layer, is_output_neuron=False):
        self.weights = []
        self.inputs = []
        self.output = None
        
        # This is used for the backpropagation update
        self.updated_weights = []
        # This is used to know how to update the weights
        self.is_output_neuron = is_output_neuron
        # This delta is used for the update at the backpropagation
        self.delta = None
        # This is used for the backpropagation update
        self.position_in_layer = position_in_layer 
        
    def attach_to_output(self, neurons):
        '''
            Helper function to store the reference of the other neurons
            To this particular neuron (used for backpropagation)
        '''
        
        self.output_neurons = []
        for neuron in neurons:
            self.output_neurons.append(neuron)
    
    def sigmoid(self, x):
        '''
            simple sigmoid function (logistic) used for the activation
        '''
        return 1 / (1 + math.exp(-x))
    
    def init_weights(self, num_input):
        '''
            This is used to setup the weights when we know how many inputs there is for
            a given neuron
        '''
        
        # Randomly initalize the weights
        for i in range(num_input):
            self.weights.append(random.uniform(0,1))
        
    def predict(self, row):
        '''
            Given a row of data it will predict what the output should be for
            this given neuron. We can have many input, but only one output for a neuron
        '''
        
        # Reset the inputs
        self.inputs = []
        
        # We iterate over the weights and the features in the given row
        activation = 0
        for weight, feature in zip(self.weights, row):
            self.inputs.append(feature)
            activation = activation + weight*feature
            
        
        self.output = self.sigmoid(activation)
        return self.output
    
        
            
    def update_neuron(self):
        '''
            Will update a given neuron weights by replacing the current weights
            with those used during the backpropagation. This need to be done at the end of the
            backpropagation
        '''
        
        self.weights = []
        for new_weight in self.updated_weights:
            self.weights.append(new_weight)
    
    def calculate_update(self, learning_rate, target):
        '''
            This function will calculate the updated weights for this neuron. It will first calculate
            the right delta (depending if this neuron is a ouput or a hidden neuron), then it will
            calculate the right updated_weights. It will not overwrite the weights yet as they are needed
            for other update in the backpropagation algorithm.
        '''
        
        if self.is_output_neuron:
            # Calculate the delta for the output
            self.delta = (self.output - target)*self.output*(1-self.output)
        else:
            # Calculate the delta
            delta_sum = 0
            # this is to know which weights this neuron is contributing in the output layer
            cur_weight_index = self.position_in_layer 
            for output_neuron in self.output_neurons:
                delta_sum = delta_sum + (output_neuron.delta * output_neuron.weights[cur_weight_index])

            # Update this neuron delta
            self.delta = delta_sum*self.output*(1-self.output)
            
            
        # Reset the update weights
        self.updated_weights = []
        
        # Iterate over each weight and update them
        for cur_weight, cur_input in zip(self.weights, self.inputs):
            gradient = self.delta*cur_input
            new_weight = cur_weight - learning_rate*gradient
            self.updated_weights.append(new_weight)
         

In [3]:
class MultiLayerPerceptron():
    '''
        We will be creating the multi-layer perceptron with only two layer:
        an input layer, a perceptrons layer and a one neuron output layer which does binary classification
    '''
    def __init__(self, num_neuron, learning_rate = 0.01, num_iteration = 100):
        # One output Neuron
        self.output_neuron = Neuron(0, is_output_neuron=True)
        
        # One hidden layer
        self.perceptrons = []
        for i in range(num_neuron):
            # Create neuron
            neuron = Neuron(i)
            # attach the output layer to this neuron
            neuron.attach_to_output([self.output_neuron])
            # Append to layer
            self.perceptrons.append(neuron)
        
        # Training parameters
        self.learning_rate = learning_rate
        self.num_iteration = num_iteration
        self.num_neuron = num_neuron
        
    
    def fit(self, X, y):
        '''
            Main training function of the neural network algorithm. This will make use of backpropagation.
            It will use stochastic gradient descent by selecting one row at random from the dataset and 
            use predict to calculate the error. The error will then be backpropagated and new weights calculated.
            Once all the new weights are calculated, the whole network weights will be updated
        '''
        num_row = len(X)
        num_feature = len(X[0]) # Here we assume that we have a rectangular matrix
        
        # Init the weights
        for neuron in self.perceptrons:
            neuron.init_weights(num_feature)
        self.output_neuron.init_weights(len(self.perceptrons))

        # Launch the training algorithm
        for i in range(self.num_iteration):
            
            # Stochastic Gradient Descent
            r_i = random.randint(0,num_row-1)
            row = X[r_i] # take the random sample from the dataset
            yhat = self.predict(row)
            target = y[r_i]
            
            # Calculate update the last layer
            self.output_neuron.calculate_update(self.learning_rate, target)
            # Calculate update the hidden layer
            for neuron in self.perceptrons:
                neuron.calculate_update(self.learning_rate, target)
            
            # Update the last layer
            self.output_neuron.update_neuron()
            # Update the hidden layer
            for neuron in self.perceptrons:
                neuron.update_neuron()
            
            # At every 100 iteration we calculate the error
            # on the whole training set
            if i % 1000 == 0:
                total_error = 0
                for r_i in range(num_row):
                    row = X[r_i]
                    yhat = self.predict(row)
                    error = (y[r_i] - yhat)
                    total_error = total_error + error**2
                mean_error = total_error/num_row
                print(f"Iteration {i} with error = {mean_error}")
        
    
    def predict(self, row):
        '''
            Prediction function that will take a row of input and give back the output
            of the whole neural network.
        '''
        
        # Gather all the activation in the hidden layer
        row.append(1) # need to add the bias
        activations = [perceptron.predict(row) for perceptron in self.perceptrons]
        
        activations.append(1) # need to add the bias
        activation = self.output_neuron.predict(activations)
        
        # Decide if we output a 1 or 0
        if activation >= 0.5:
            return 1.0
        return 0.0

In [8]:
# XOR function (one or the other but not both)
X = [[0,0], [0,1], [1,0], [1,1]]
y = [0, 1, 1, 0]

num_neuron = 10
num_iteration = 150000
clf = MultiLayerPerceptron(num_neuron, num_iteration = num_iteration)
clf.fit(X,y)

Iteration 0 with error = 0.5
Iteration 1000 with error = 0.5
Iteration 2000 with error = 0.5
Iteration 3000 with error = 0.5
Iteration 4000 with error = 0.5
Iteration 5000 with error = 0.5
Iteration 6000 with error = 0.25
Iteration 7000 with error = 0.5
Iteration 8000 with error = 0.5
Iteration 9000 with error = 0.5
Iteration 10000 with error = 0.5
Iteration 11000 with error = 0.5
Iteration 12000 with error = 0.25
Iteration 13000 with error = 0.5
Iteration 14000 with error = 0.75
Iteration 15000 with error = 0.5
Iteration 16000 with error = 0.25
Iteration 17000 with error = 0.75
Iteration 18000 with error = 0.5
Iteration 19000 with error = 0.25
Iteration 20000 with error = 0.25
Iteration 21000 with error = 0.25
Iteration 22000 with error = 0.5
Iteration 23000 with error = 0.25
Iteration 24000 with error = 0.25
Iteration 25000 with error = 0.25
Iteration 26000 with error = 0.5
Iteration 27000 with error = 0.25
Iteration 28000 with error = 0.5
Iteration 29000 with error = 0.25
Iteration 

In [12]:
clf.predict([1,1])

0.0